In [ ]:
import pandas as pd

def save_ids_to_txt_file(df, file_path):
    """
    Save the IDs from the DataFrame to a text file, one per line.

    Args:
        df (pd.DataFrame): DataFrame containing an 'id' column.
        file_path (str): Path to the output text file.
    """
    with open(file_path, 'w') as f:
        for index, row in df.iterrows():
            f.write(f"{row['id']}\n")

# Load the CSV file containing cell data
df = pd.read_csv('data/h01.csv')

# Filter for neurons using the 'data' column
df_neurons = df[df['data'].str.contains('#neuron')]
print("Neurons:", len(df_neurons))
save_ids_to_txt_file(df_neurons, 'neurons.ids')

# Filter for pyramidal neurons
df_pyramidal = df_neurons[df_neurons['data'].str.contains('#pyramidal ')]
print("Pyramidals:", len(df_pyramidal))
save_ids_to_txt_file(df_pyramidal, 'pyramidal.ids')

# Save pyramidal neuron IDs by cortical layer
for i in [1, 2, 3, 4, 5, 6]:
    df_pyramidal_layer = df_neurons[df_neurons['data'].str.contains(f'#L{i}')]
    print(f"Pyramidals: L{i}", len(df_pyramidal_layer))
    save_ids_to_txt_file(df_pyramidal_layer, f'pyramidal_L{i}.ids')

# Filter for interneurons
df_interneuron = df_neurons[df_neurons['data'].str.contains('#interneuron ')]
print("Interneurons:", len(df_interneuron))
save_ids_to_txt_file(df_interneuron, 'interneurons.ids')

# Save interneuron IDs by cortical layer
for i in [1, 2, 3, 4, 5, 6]:
    df_interneuron_layer = df_interneuron[df_interneuron['data'].str.contains(f'#L{i}')]
    print(f"Interneurons: L{i}", len(df_interneuron_layer))
    save_ids_to_txt_file(df_interneuron_layer, f'interneurons_L{i}.ids')


In [65]:
import caveclient
url = "https://global.brain-wire-test.org/"
auth = caveclient.auth.AuthClient(server_address=url)
auth.setup_token(make_new=True)

New Tokens need to be acquired by hand. Please follow the following steps:
                1) Go to: https://global.brain-wire-test.org//auth/api/v1/create_token to create a new token.
                2) Log in with your Google credentials and copy the token shown afterward.
                3a) Save it to your computer with: client.auth.save_token(token="PASTE_YOUR_TOKEN_HERE")
                or
                3b) Set it for the current session only with client.auth.token = "PASTE_YOUR_TOKEN_HERE"
                Note: If you need to save or load multiple tokens, please read the documentation for details.
                Warning! Creating a new token by finishing step 2 will invalidate the previous token!


Opening in existing browser session.


In [67]:
auth_token = "be1bde27490bb4f37580cc370ba9068a"
auth.save_token(token=auth_token, overwrite=True)

In [ ]:
from caveclient import CAVEclient
client = CAVEclient()
try:
    client.auth.save_token(token=auth_token)
except :
    print("Token is already there!")

Token is already there!
8841fffdac923d4cee7b07fe82e11987


In [ ]:
import os 
import cloudvolume

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def get_cave_client(dataset):
    """
    Returns a CAVEclient for the given dataset.
    """
    client = caveclient.CAVEclient(dataset, auth_token=auth_token)
    print(client.info.get_datastack_info())
    return client

def get_cloud_volume(path):
    """
    Returns a CloudVolume object for the given path.
    """
    print("Getting the cloud volume")
    return cloudvolume.CloudVolume(path, use_https=True, progress=True)

def load_ids_from_file(file_path):
    """
    Loads a list of IDs from a file (one per line).
    """
    with open(file_path, 'r') as f:
        ids = [l.strip('\n') for l in f]
    return ids

def download_cell_from_cloud_volume(cloud_volume, cell_id, output_directory):    
    """
    Downloads a mesh for a single cell_id from the cloud volume and saves as .obj.
    """
    try:
        mesh = cloud_volume.mesh.get(cell_id)[cell_id]
        with open(f'{output_directory}/{cell_id}.obj', 'wb') as f:
            f.write(mesh.to_obj())
    except Exception as e:
        print(f"Error in downloading the cell {cell_id}")
        print(e)
        return None

def download_cells_from_cloud_volume(cloud_volume, cell_ids, output_directory):
    """
    Downloads meshes for a list of cell_ids from the cloud volume.
    """
    for cell_id in cell_ids:
        if int(cell_id) > 0:
            print(f'Loading [{cell_id}]')
            download_cell_from_cloud_volume(cloud_volume, int(cell_id), output_directory)

def download_cells_from_cloud_volume_parallel(cloud_volume, cell_ids, output_directory, n_cores=8):
    """
    Downloads meshes for a list of cell_ids in parallel.
    """
    from joblib import Parallel, delayed
    commands = [[int(i), output_directory] for i in cell_ids]
    Parallel(n_jobs=int(n_cores))(delayed(download_cell_from_cloud_volume)(
        cloud_volume, i[0], i[1]) for i in commands)


In [ ]:
ids_file = "/scratch/h01-data/ids/pyramidal_L5.ids"
output_directory = "/hdd1/data/h01/pyramidal_L5"

ids = load_ids_from_file(ids_file)
print("Number of ids: ", len(ids))

# Create the output directory
create_dir(output_directory)

# Craete the cloud volume
cloud_volume_path = "precomputed://gs://h01-release/data/20210601/c3"
cloud_volume = get_cloud_volume(cloud_volume_path)

# Download the cells from the cloud volume
download_cells_from_cloud_volume_parallel(cloud_volume, ids, output_directory)